In [4]:
import scipy.io
from scipy import signal
from scipy import stats
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt

In [5]:
# Data importation : ECG&EEG extraction 
# Doubt about ECG, Resp1 & Resp2 data, are they really that ?


Files = [r'SOP01',
         r'SOP02',
         r'SOP03',
         r'SOP04',
         r'SOP05',
         r'SOP06', # Files = [
         r'SOP07',
         r'SOP08',
         r'SOP09',
         r'SOP10',
         r'SOP11',
         r'SOP12',
         r'SOP13',
         r'SOP14',
         r'SOP15',
         r'SOP16',
         r'SOP17',
         r'SOP18',
         r'SOP19',
         r'SOP20']

Files_QoE = ['SOP1',
             'SOP2',
             'SOP3',
             'SOP4',
             'SOP5',
             'SOP6', # Files_QoE = [
             'SOP7',
             'SOP8',
             'SOP9',
             'SOP10',
             'SOP11',
             'SOP12',
             'SOP13',
             'SOP14',
             'SOP15',
             'SOP16',
             'SOP17',
             'SOP18',
             'SOP19',
             'SOP20']

In [6]:

def data_extraction(Files, Files_QoE):
    data_files = [r'_SA.seg.mat',r'_SB.seg.mat',r'_SC.seg.mat']
    data_streams = [r'_SAseg',r'_SBseg',r'_SCseg']
    data_EEG = []
    data_impedance = []
    data_Resp1 = []
    data_Resp2 = []
    data_ECG = []
    
    print("--- Extracting data ---")
    
    for k in Files:
        data_EEG_session = []
        data_ECG_session = []
        data_Resp1_session = []
        data_Resp2_session = []
        data_impedance_temp = []
        for i in range(len(data_files)):
            print("--- Loading ", k+data_files[i], " ---")
            path_data = r'/o'[0] + k + r'/o'[0] + k + data_files[i] 
            data_temp = scipy.io.loadmat(r'../EEG_ECG_Respiration_data' + path_data)

            data_EEG_temp = []
            print('Verif EEG:', np.transpose(data_temp[k+data_streams[i]][0]).shape)
            data_EEG_temp += [np.transpose(data_temp[k+data_streams[i]][0][0])] # 1er clip
            data_EEG_temp += [np.transpose(data_temp[k+data_streams[i]][0][1])] # 2nd clip
            data_EEG_temp += [np.transpose(data_temp[k+data_streams[i]][0][2])]
            data_EEG_temp += [np.transpose(data_temp[k+data_streams[i]][0][3])]
            data_EEG_temp += [np.transpose(data_temp[k+data_streams[i]][0][4])]
            data_EEG_temp += [np.transpose(data_temp[k+data_streams[i]][0][5])]
            data_EEG_temp += [np.transpose(data_temp[k+data_streams[i]][0][6])]
            data_EEG_temp += [np.transpose(data_temp[k+data_streams[i]][0][7])]
            data_EEG_temp += [np.transpose(data_temp[k+data_streams[i]][0][8])] # 9ème clip

            data_EEG_session += [data_EEG_temp]
            
            print('Verif Impedence:', np.transpose(data_temp['Impedances_0']).shape)
            data_impedance_temp += [data_temp['Impedances_0']]
            
            print("--- Loaded ---")

        data_EEG += [data_EEG_session]

    print("Should be 16 :",len(data_EEG))
    print("Should be 3 :",len(data_EEG[0]))
    print("Should be 9 :",len(data_EEG[0][0]))
    print("Should be 17500, 257 :", data_EEG[0][0][0].shape)

    
    # Selection des electrodes EEG (source EduCovid) : Fp1, Fp2, T3, T4, P3, P4, O1, O2
    
    print("--- Selecting electrods ---")
    selected_EEG = []
    ses = ["A","B","C"]
    for k in range(len(data_EEG)): # Par sujet
        print("------ Sujet n°%s ------" %(k+1))
        EEG_sujet = []
        for i in range(len(data_EEG[0])): # Par session 
            print("--- Session %s ---" %ses[i])
            EEG_session = []
            for j in range(len(data_EEG[0][0])): # Par video
                EEG_video = []
                for l in range(data_EEG[0][0][0].shape[0]): #Recup' de toute l'acquisition
                    Fp1 = data_EEG[k][i][j][l][33]
                    Fp2 = data_EEG[k][i][j][l][19]
                    T3 = data_EEG[k][i][j][l][69]
                    T4 = data_EEG[k][i][j][l][202]
                    P3 = data_EEG[k][i][j][l][86] 
                    P4 = data_EEG[k][i][j][l][162]
                    O1 = data_EEG[k][i][j][l][124]
                    O2 = data_EEG[k][i][j][l][149]
                    #EEG_video += [data_EEG[k][i][j][l][:-1]]  #Paramétre que je teste en commentaire à la base. Le nombre de colonnes EEG est 256
                    EEG_video += [[Fp1,Fp2,T3,T4,P3,P4,O1,O2]] #Paramétre de base. Le nombre de colonnes EEG est 8
                EEG_session += [EEG_video]
            EEG_sujet += [EEG_session]
        selected_EEG += [EEG_sujet]

    print("--- Done ---")
    print("Should be 16 : ",len(selected_EEG)) # Sujet
    print("Should be 3 : ",len(selected_EEG[0])) # Session
    print("Should be 9 : ",len(selected_EEG[0][0])) # Videos
    print("Should be 17500 : ",len(selected_EEG[0][0][0])) # lignes EEG
    print("Should be 8 : ",len(selected_EEG[0][0][0][0])) # colonnes EEG
    
    return selected_EEG

selected_EEG = data_extraction(Files,Files_QoE)

--- Extracting data ---
--- Loading  SOP01_SA.seg.mat  ---
Verif EEG: (9,)
Verif Impedence: (1, 257)
--- Loaded ---
--- Loading  SOP01_SB.seg.mat  ---
Verif EEG: (9,)
Verif Impedence: (1, 257)
--- Loaded ---
--- Loading  SOP01_SC.seg.mat  ---
Verif EEG: (9,)
Verif Impedence: (1, 257)
--- Loaded ---
--- Loading  SOP02_SA.seg.mat  ---
Verif EEG: (9,)
Verif Impedence: (1, 257)
--- Loaded ---
--- Loading  SOP02_SB.seg.mat  ---
Verif EEG: (9,)
Verif Impedence: (1, 257)
--- Loaded ---
--- Loading  SOP02_SC.seg.mat  ---
Verif EEG: (9,)
Verif Impedence: (1, 257)
--- Loaded ---
--- Loading  SOP03_SA.seg.mat  ---
Verif EEG: (9,)
Verif Impedence: (1, 257)
--- Loaded ---
--- Loading  SOP03_SB.seg.mat  ---
Verif EEG: (9,)
Verif Impedence: (1, 257)
--- Loaded ---
--- Loading  SOP03_SC.seg.mat  ---
Verif EEG: (9,)
Verif Impedence: (1, 257)
--- Loaded ---
--- Loading  SOP04_SA.seg.mat  ---
Verif EEG: (9,)
Verif Impedence: (1, 257)
--- Loaded ---
--- Loading  SOP04_SB.seg.mat  ---
Verif EEG: (9,)
Verif

In [7]:
# Filter EEG_ECG

def filtrageNotch(sig, fRemove, sampleFreq, Quality=30):
    # Design
    b, a = signal.iirnotch(fRemove, Quality, sampleFreq)
    # Apply
    signalFiltered = signal.filtfilt(b, a, sig)
    return signalFiltered

In [8]:
def filtragePasseBande(sig, fcHaut, fcBas, sampleFreq):
    # Fréquence d'échantillonnage
    fe = sampleFreq  # Hz
    
    # Fréquence de nyquist
    f_nyq = fe / 2.  # Hz
    
    # Fréquence de coupure
    fc1 = fcBas  # Hz
    fc2 = fcHaut  # Hz
    
    # Préparation du filtre de Butterworth en passe bande
    sos = signal.butter(4, [fc1, fc2], fs=sampleFreq, btype='band',output='sos') # 'bandpass',
    signal_bp = signal.sosfilt(sos, sig)

#     b, a = signal.butter(4, fc1/f_nyq, 'low', analog=False)    #Aucuns changements particuliers remarqués
#     signal_pb = signal.filtfilt(b, a, sig)
    
#     b, a = signal.butter(4, fc2/f_nyq, 'high', analog=False)
#     signal_bp = signal.filtfilt(b, a, signal_pb)
    
    return signal_bp

In [9]:
## Filtering & ESD calculation
def Filtering_EEG(EEG):
    final_EEG = []
    ses = ["A","B","C"]
    for k in range(len(EEG)): # Par sujet
        print("------ Sujet n°%s ------" %(k+1))
        sujet_EEG = []
        for i in range(len(EEG[0])): # Par session
            print("--- Session %s ---" %ses[i])
            session_EEG = []
            for j in range(len(EEG[0][0])): # Par video
                # print("--- Video n°%s ---" %(j+1))
                video_EEG = []
#                 allEEG = []
                for x in range(len(EEG[0][0][0][0])): # par electrode
                    electrode = []
                    for l in range(len(EEG[0][0][0])-2500): # Toute la session
                        electrode += [EEG[k][i][j][2500+l][x]]

                    # Filtering 
                    notched = filtrageNotch(electrode,50,250)
                    electrode_bp = filtragePasseBande(notched,47,1,250)

                    video_EEG += [electrode_bp]

                session_EEG += [np.transpose(video_EEG)]
            sujet_EEG += [session_EEG]
        final_EEG += [sujet_EEG]
    return final_EEG

In [10]:
print("--- Filtering EEG ---")
EEG_filtered = Filtering_EEG(selected_EEG)

print("--- Done ---")
print("Should be 20 : ",len(EEG_filtered)) # Sujet
print("Should be 3 : ",len(EEG_filtered[0])) # Session
print("Should be 9 : ",len(EEG_filtered[0][0])) # Videos
print("Should be 15000 : ",len(EEG_filtered[0][0][0])) # lignes EEG
print("Should be 256 : ",len(EEG_filtered[0][0][0][0])) # colonnes EEG/ nombres d'éléctrodes utilisés


--- Filtering EEG ---
------ Sujet n°1 ------
--- Session A ---
--- Session B ---
--- Session C ---
------ Sujet n°2 ------
--- Session A ---
--- Session B ---
--- Session C ---
------ Sujet n°3 ------
--- Session A ---
--- Session B ---
--- Session C ---
------ Sujet n°4 ------
--- Session A ---
--- Session B ---
--- Session C ---
------ Sujet n°5 ------
--- Session A ---
--- Session B ---
--- Session C ---
------ Sujet n°6 ------
--- Session A ---
--- Session B ---
--- Session C ---
------ Sujet n°7 ------
--- Session A ---
--- Session B ---
--- Session C ---
------ Sujet n°8 ------
--- Session A ---
--- Session B ---
--- Session C ---
------ Sujet n°9 ------
--- Session A ---
--- Session B ---
--- Session C ---
------ Sujet n°10 ------
--- Session A ---
--- Session B ---
--- Session C ---
------ Sujet n°11 ------
--- Session A ---
--- Session B ---
--- Session C ---
------ Sujet n°12 ------
--- Session A ---
--- Session B ---
--- Session C ---
------ Sujet n°13 ------
--- Session A 

In [11]:
# Welch
#fournit un estimateur consistant de la densité spectrale de puissance.

# EEG features extraction on 0.5 s window

def GetWavebands(sig):
    compt = 0
#     max_len = len(sig)
    for k in range(len(sig)):
        if (sig[k]==0):
            compt += 1
        
    if (compt >= 1):
        return [0,0,0,0,0,0,0,0,0,0]
    
    # FFT
    y = sig
    # yf = scipy.fftpack.fft(y)
    xf, yf = signal.welch(y,fs=250,nperseg=(250))
    # xf,yf = plt.psd(y,Fs=250)
    
    # Define delta lower and upper limits
    Dlow, Dhigh = 1, 4
    # Find intersecting values in frequency vector
    idx_delta = np.logical_and(xf >= Dlow, xf <= Dhigh)
    
    # Define theta lower and upper limits
    Tlow, Thigh = 4, 8
    # Find intersecting values in frequency vector
    idx_theta = np.logical_and(xf >= Tlow, xf <= Thigh)
    
    # Define Alpha lower and upper limits
    low, high = 8, 12
    # Find intersecting values in frequency vector
    idx_alpha = np.logical_and(xf >= low, xf <= high)
    
    # Define Beta lower and upper limits
    Blow, Bhigh = 12, 30
    # Find intersecting values in frequency vector
    idx_beta = np.logical_and(xf >= Blow, xf <= Bhigh)
    
    # Define gamma lower and upper limits
    Glow, Ghigh = 30, 60
    # Find intersecting values in frequency vector
    idx_gamma = np.logical_and(xf >= Glow, xf <= Ghigh)
    
    
    # Calcul PSD
    delta = []
    theta = []
    alpha = []
    beta = []
    gamma = []
    
    for k in range(len(xf)):
#         if (len(idx_alpha) == 0) or (len(idx_beta) == 0) or (len(idx_theta) == 0):
#            break
        if (idx_delta[k]==True):
            delta += [float(yf[k])]
        elif (idx_theta[k]==True):
            theta += [float(yf[k])]
        elif (idx_alpha[k]==True):
            alpha += [float(yf[k])]
        elif (idx_beta[k]==True):
            beta += [float(yf[k])]
        elif (idx_gamma[k]==True):
            gamma += [float(yf[k])]
    
    # Calcul DE
    sig_delta = filtragePasseBande(sig,4,1,250)
    DE_Delta = stats.differential_entropy(sig_delta)
    sig_theta = filtragePasseBande(sig,8,4,250)
    DE_Theta = stats.differential_entropy(sig_theta)
    sig_alpha = filtragePasseBande(sig,12,4,250)
    DE_Alpha = stats.differential_entropy(sig_alpha)
    sig_beta = filtragePasseBande(sig,30,12,250)
    DE_Beta = stats.differential_entropy(sig_beta)
    sig_gamma = filtragePasseBande(sig,60,30,250)
    DE_Gamma = stats.differential_entropy(sig_gamma)
    
    return [float(np.mean(delta)),float(np.mean(theta)),float(np.mean(alpha)),float(np.mean(beta)),float(np.mean(gamma)), 
            DE_Delta, DE_Theta, DE_Alpha, DE_Beta, DE_Gamma]

In [12]:
# EEG features extraction on 3s with 50% overlap window
## ESD calculation
def EEG_feat(EEG,window):
    
    final_EEG = []
    win = int((window*250)/2)
    ses = ["A","B","C"]
    start_time = time.time()
    for k in range(len(EEG)): # Par sujet
        print("------ Sujet n°%s ------" %(k+1))
        sujet_EEG = []
        for i in range(len(EEG[0])): # Par session
            print("--- Session %s ---" %ses[i])
            session_EEG = []
            for j in range(len(EEG[0][0])): # Par video
                video_st = time.time()
                video_EEG = []                    
                for l in range(int(len(EEG[0][0][0])/win)):
                    features = []
                    # Calculate ESD per window
                    for x in range(len(EEG[0][0][0][0])): # par electrode
                        electrode = []
                        for x2 in range(len(EEG[0][0][0])): # Toute la session (à partir de 2(window*250) points)
                            electrode += [EEG[k][i][j][x2][x]]

                        M_delta,M_theta,M_alpha,M_beta,M_gamma,DE_Delta,DE_Theta,DE_Alpha,DE_Beta,DE_Gamma = GetWavebands(electrode[l*win:(l*win)+(window*250)])

                        features += [M_delta,M_theta,M_alpha,M_beta,M_gamma,DE_Delta,DE_Theta,DE_Alpha,DE_Beta,DE_Gamma]
                    video_EEG += [features]
                print("Video done in :", time.time()-video_st)
            
                df = pd.DataFrame(video_EEG)
                if (k<9):
                    name = r'\SOP0'+str(k+1)+'_'+ses[i]+'_'+str(j+1)+'.csv'
                else:
                    name = r'\SOP'+str(k+1)+'_'+ses[i]+'_'+str(j+1)+'.csv'
                df.to_csv(r'../data_EEG/3seconds'+name)
                
                session_EEG += [video_EEG]
            sujet_EEG += [session_EEG]
        final_EEG += [sujet_EEG]
    
    print("--- Done in %s s ---" %(time.time()-start_time))
    print("Should be 20 : ",len(final_EEG)) # Sujet
    print("Should be 3 : ",len(final_EEG[0])) # Session
    print("Should be 9 : ",len(final_EEG[0][0])) # Videos
    print("Should be 24 : ",len(final_EEG[0][0][0])) # lignes EEG
    print("Should be 80 : ",len(final_EEG[0][0][0][0])) # colonnes EEG
    
    return final_EEG

EEG_features = EEG_feat(EEG_filtered,3)
print(np.asarray(EEG_features).shape)

------ Sujet n°1 ------
--- Session A ---
Video done in : 11.49489450454712
Video done in : 11.055986881256104
Video done in : 11.745972633361816
Video done in : 11.396822929382324
Video done in : 11.395074844360352
Video done in : 11.262858390808105
Video done in : 11.265693664550781
Video done in : 11.579806327819824
Video done in : 11.671212434768677
--- Session B ---
Video done in : 11.361384868621826
Video done in : 11.330013751983643
Video done in : 11.007031679153442
Video done in : 11.25508165359497
Video done in : 11.61991262435913
Video done in : 11.657395124435425
Video done in : 10.976256847381592
Video done in : 11.592300415039062
Video done in : 10.404683589935303
--- Session C ---
Video done in : 11.68135929107666
Video done in : 11.740121126174927
Video done in : 11.73922610282898
Video done in : 11.479456186294556
Video done in : 10.802973985671997
Video done in : 11.22783875465393
Video done in : 11.099031209945679
Video done in : 10.216634750366211
Video done in : 11

Video done in : 13.200303316116333
Video done in : 12.515210628509521
Video done in : 12.505438327789307
Video done in : 12.451095819473267
Video done in : 12.580169200897217
Video done in : 12.691302299499512
Video done in : 12.569993019104004
Video done in : 12.516830921173096
--- Session B ---
Video done in : 12.43511152267456
Video done in : 12.721433639526367
Video done in : 12.55132246017456
Video done in : 12.530088901519775
Video done in : 12.349526166915894
Video done in : 12.353224277496338
Video done in : 12.799541234970093
Video done in : 12.569290161132812
Video done in : 12.326827049255371
--- Session C ---
Video done in : 12.504732370376587
Video done in : 12.506155490875244
Video done in : 12.526550769805908
Video done in : 12.416939735412598
Video done in : 12.336026191711426
Video done in : 12.297742128372192
Video done in : 12.67290210723877
Video done in : 12.610529899597168
Video done in : 12.51691484451294
------ Sujet n°10 ------
--- Session A ---
Video done in :

Video done in : 8.868422985076904
Video done in : 8.999907732009888
Video done in : 8.971652746200562
Video done in : 9.054885149002075
Video done in : 8.890035629272461
Video done in : 8.970069646835327
--- Session B ---
Video done in : 9.267914056777954
Video done in : 9.060187339782715
Video done in : 9.204703092575073
Video done in : 9.008536577224731
Video done in : 9.147329568862915
Video done in : 9.000363111495972
Video done in : 9.07363247871399
Video done in : 9.104698419570923
Video done in : 8.958338737487793
--- Session C ---
Video done in : 9.235932111740112
Video done in : 8.892738342285156
Video done in : 9.104735374450684
Video done in : 9.189590215682983
Video done in : 9.263315677642822
Video done in : 9.119064569473267
Video done in : 9.03879165649414
Video done in : 9.264713287353516
Video done in : 9.102159023284912
------ Sujet n°18 ------
--- Session A ---
Video done in : 9.123298645019531
Video done in : 9.079894065856934
Video done in : 8.979375839233398
Video

In [13]:
# DWT

import pywt

wavelet = pywt.Wavelet('db4')
print(wavelet)
print(wavelet.dec_lo)

# pywt.wavedec(data, 'db4', level=5)

Wavelet db4
  Family name:    Daubechies
  Short name:     db
  Filters length: 8
  Orthogonal:     True
  Biorthogonal:   True
  Symmetry:       asymmetric
  DWT:            True
  CWT:            False
[-0.010597401785069032, 0.0328830116668852, 0.030841381835560764, -0.18703481171909309, -0.027983769416859854, 0.6308807679298589, 0.7148465705529157, 0.2303778133088965]


In [14]:
# EEG features extraction on 0.5 s window

def GetWavebands(sig):
    compt = 0
#     max_len = len(sig)
    for k in range(len(sig)):
        if (sig[k]==0):
            compt += 1
        
    if (compt >= 1):
        return [0,0,0,0,0,0,0,0,0,0]
    
    delta, theta, alpha, beta, gamma, noise = pywt.wavedec(sig, wavelet, level=5)
#     print('Test sig :', len(decomposed))
#     print('Test sig :', len(decomposed[0]))
#     print('Test sig :', len(decomposed[0][0]))
    
#     print('Test sig :', len(cD))
#     print('Test sig :', len(cD[0]))
    
    
    
    # Calcul DE
    sig_delta = filtragePasseBande(sig,4,1,250)
    DE_Delta = stats.differential_entropy(sig_delta)
    sig_theta = filtragePasseBande(sig,8,4,250)
    DE_Theta = stats.differential_entropy(sig_theta)
    sig_alpha = filtragePasseBande(sig,16,8,250)
    DE_Alpha = stats.differential_entropy(sig_alpha)
    sig_beta = filtragePasseBande(sig,32,16,250)
    DE_Beta = stats.differential_entropy(sig_beta)
    sig_gamma = filtragePasseBande(sig,64,32,250)
    DE_Gamma = stats.differential_entropy(sig_gamma)
    
    return [float(np.mean(delta)),float(np.mean(theta)),float(np.mean(alpha)),float(np.mean(beta)),float(np.mean(gamma)), 
            DE_Delta, DE_Theta, DE_Alpha, DE_Beta, DE_Gamma]

In [15]:
# EEG features extraction on 3s with 50% overlap window
## ESD calculation
def EEG_feat(EEG,window):
    
    final_EEG = []
    win = int((window*250)/2)
    ses = ["A","B","C"]
    start_time = time.time()
    for k in range(len(EEG)): # Par sujet
        print("------ Sujet n°%s ------" %(k+1))
        sujet_EEG = []
        for i in range(len(EEG[0])): # Par session
            print("--- Session %s ---" %ses[i])
            session_EEG = []
            for j in range(len(EEG[0][0])): # Par video
                video_st = time.time()
                video_EEG = []                    
                for l in range(int(len(EEG[0][0][0])/win)):
                    features = []
                    # Calculate ESD per window
                    for x in range(len(EEG[0][0][0][0])): # par electrode
                        electrode = []
                        for x2 in range(len(EEG[0][0][0])): # Toute la session (à partir de 2(window*250) points)
                            electrode += [EEG[k][i][j][x2][x]]

                        M_delta,M_theta,M_alpha,M_beta,M_gamma,DE_Delta,DE_Theta,DE_Alpha,DE_Beta,DE_Gamma = GetWavebands(electrode[l*win:(l*win)+(window*250)])

                        features += [M_delta,M_theta,M_alpha,M_beta,M_gamma,DE_Delta,DE_Theta,DE_Alpha,DE_Beta,DE_Gamma]
                    video_EEG += [features]
                print("Video done in :", time.time()-video_st)
            
                df = pd.DataFrame(video_EEG)
                if (k<9):
                    name = r'\SOP0'+str(k+1)+'_'+ses[i]+'_'+str(j+1)+'.csv'
                else:
                    name = r'\SOP'+str(k+1)+'_'+ses[i]+'_'+str(j+1)+'.csv'
                df.to_csv(r'../data_EEG/3seconds'+name)
                
                session_EEG += [video_EEG]
            sujet_EEG += [session_EEG]
        final_EEG += [sujet_EEG]
    
    print("--- Done in %s s ---" %(time.time()-start_time))
    print("Should be 20 : ",len(final_EEG)) # Sujet
    print("Should be 3 : ",len(final_EEG[0])) # Session
    print("Should be 9 : ",len(final_EEG[0][0])) # Videos
    print("Should be 24 : ",len(final_EEG[0][0][0])) # lignes EEG
    print("Should be 80 : ",len(final_EEG[0][0][0][0])) # colonnes EEG
    
    return final_EEG

EEG_features = EEG_feat(EEG_filtered,3)
print(np.asarray(EEG_features).shape)

------ Sujet n°1 ------
--- Session A ---
Video done in : 7.242894172668457
Video done in : 7.046881198883057
Video done in : 7.31397557258606
Video done in : 7.091581106185913
Video done in : 1781.7002353668213
Video done in : 5.764601707458496
Video done in : 3.0184309482574463
Video done in : 61649.83616614342
Video done in : 8.256697416305542
--- Session B ---
Video done in : 6.64243483543396
Video done in : 4.490563154220581
Video done in : 3.823758840560913
Video done in : 4.153024911880493
Video done in : 4.08886194229126
Video done in : 3.2748281955718994
Video done in : 3.3941352367401123
Video done in : 3.3952085971832275
Video done in : 4.431553363800049
--- Session C ---
Video done in : 3.9325478076934814
Video done in : 4.297516107559204
Video done in : 3.808704137802124
Video done in : 4.478354454040527
Video done in : 3.3934953212738037
Video done in : 3.4807658195495605
Video done in : 3.194669008255005
Video done in : 3.4920482635498047
Video done in : 4.42501926422119

Video done in : 3.4563252925872803
Video done in : 3.3851184844970703
Video done in : 3.326631784439087
Video done in : 3.7662971019744873
Video done in : 3.8054044246673584
--- Session B ---
Video done in : 3.9174888134002686
Video done in : 3.5316519737243652
Video done in : 3.4710700511932373
Video done in : 3.5220367908477783
Video done in : 3.4387292861938477
Video done in : 3.622314214706421
Video done in : 3.5174663066864014
Video done in : 3.334794044494629
Video done in : 3.6707658767700195
--- Session C ---
Video done in : 3.4498910903930664
Video done in : 3.409623146057129
Video done in : 3.5003256797790527
Video done in : 3.3257648944854736
Video done in : 3.4871978759765625
Video done in : 3.192063331604004
Video done in : 3.6540908813476562
Video done in : 3.408687114715576
Video done in : 3.3903415203094482
------ Sujet n°10 ------
--- Session A ---
Video done in : 3.3544836044311523
Video done in : 3.273530960083008
Video done in : 3.1544253826141357
Video done in : 3.

Video done in : 2.3272902965545654
--- Session B ---
Video done in : 2.3612377643585205
Video done in : 2.3597569465637207
Video done in : 2.3224380016326904
Video done in : 2.367039203643799
Video done in : 2.2764780521392822
Video done in : 2.521768808364868
Video done in : 2.4208014011383057
Video done in : 2.4053914546966553
Video done in : 2.3927183151245117
--- Session C ---
Video done in : 2.3522441387176514
Video done in : 2.3861658573150635
Video done in : 2.3465137481689453
Video done in : 2.3382973670959473
Video done in : 2.478583574295044
Video done in : 2.4847798347473145
Video done in : 2.3435592651367188
Video done in : 2.4103891849517822
Video done in : 2.3479537963867188
------ Sujet n°18 ------
--- Session A ---
Video done in : 2.3393728733062744
Video done in : 2.3399341106414795
Video done in : 2.4253034591674805
Video done in : 2.472280979156494
Video done in : 2.420386552810669
Video done in : 2.404715061187744
Video done in : 2.3873441219329834
Video done in : 2